In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer

2025-05-28 13:20:44.357073: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 13:20:44.365950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748438444.375260   18663 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748438444.378431   18663 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748438444.386488   18663 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [31]:
class BaseComponent():
    def forward(self, input_data):
        """Forward pass through the component."""
        pass

    def backward(self, gradient):
        """Backward pass through the component."""
        pass

class RNNCell(BaseComponent):
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size

        # Initialize weights and biases
        self.W_x = np.random.randn(hidden_size, input_size) * 0.01
        self.W_h = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_y = np.random.randn(output_size, hidden_size) * 0.01
        self.b_h = np.zeros((hidden_size, 1))
        self.b_y = np.zeros((output_size, 1))

    def forward(self, x_t, h_prev):
        """Compute the hidden state and output for a single time step."""
        self.x_t = x_t
        self.h_prev = h_prev

        # Hidden state update
        self.h_t = np.tanh(np.dot(self.W_x, x_t) + np.dot(self.W_h, h_prev) + self.b_h)

        # Output calculation
        self.y_t = self.softmax(np.dot(self.W_y, self.h_t) + self.b_y)

        return self.h_t, self.y_t

    def backward(self, dy, dh_next):
        """Compute gradients for weights and biases."""
        dW_y = np.dot(dy, self.h_t.T)
        db_y = dy

        dh = np.dot(self.W_y.T, dy) + dh_next
        dh_raw = (1 - self.h_t ** 2) * dh

        dW_x = np.dot(dh_raw, self.x_t.T)
        dW_h = np.dot(dh_raw, self.h_prev.T)
        db_h = dh_raw

        dh_prev = np.dot(self.W_h.T, dh_raw)

        # Store gradients
        self.grads = {
            "W_x": dW_x,
            "W_h": dW_h,
            "W_y": dW_y,
            "b_h": db_h,
            "b_y": db_y,
        }

        return dh_prev

    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum(axis=0, keepdims=True)

class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.cell = RNNCell(input_size, hidden_size, output_size)

    def forward(self, X):
        """Process the entire sequence."""
        self.h_states = []
        self.outputs = []

        h_t = np.zeros((self.cell.hidden_size, 1))  # Initial hidden state

        for x_t in X:
            h_t, y_t = self.cell.forward(x_t, h_t)
            self.h_states.append(h_t)
            self.outputs.append(y_t)

        return self.outputs

    def backward(self, dY):
        """Backpropagate through the entire sequence."""
        dh_next = np.zeros_like(self.h_states[0])
        for t in reversed(range(len(self.h_states))):
            dh_next = self.cell.backward(dY[t], dh_next)

class Trainer:
    def __init__(self, rnn, learning_rate=0.01):
        self.rnn = rnn
        self.learning_rate = learning_rate

    def compute_loss(self, y_pred, y_true):
        """Compute cross-entropy loss."""
        return -np.sum(y_true * np.log(y_pred + 1e-8))

    def update_parameters(self):
        """Update weights and biases using gradients."""
        for param, grad in self.rnn.cell.grads.items():
            setattr(self.rnn.cell, param, getattr(self.rnn.cell, param) - self.learning_rate * grad)

    def train(self, X, Y, epochs):
        """Train the RNN on a dataset of tokenized sentences."""
        for epoch in range(epochs):
            total_loss = 0
    
            data = list(zip(X, Y))
            np.random.shuffle(data)
            X, Y = zip(*data)
    
            for x_seq, y_true in zip(X, Y):
                # Forward pass
                y_pred = self.rnn.forward(x_seq)[-1]  # Use the final output for classification
    
                # Compute loss
                loss = self.compute_loss(y_pred, y_true)  # Directly compute loss for scalars
                total_loss += loss
    
                # Backward pass
                dY = y_pred - y_true  # Gradient of loss w.r.t. prediction
                self.rnn.backward([dY])  # Wrap dY in a list to match expected input
    
                # Update parameters
                self.update_parameters()
    
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(X)}")

In [32]:
df_traindata = pd.read_csv("../data/train.csv")

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_traindata['sentiment'].tolist())

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df_traindata['text'].tolist())
sequences = tokenizer.texts_to_sequences(df_traindata['text'].tolist())

In [33]:
trainer = Trainer(RNN(100, 128, 1))

trainer.train(sequences, labels, 10)

IndexError: list index out of range